# Window functions (OVER)

Check following sources:

- [SQL window functions](https://mode.com/sql-tutorial/sql-window-functions) tutorial on mode.com.
- [Window functions](https://www.postgresql.org/docs/9.1/tutorial-window.html) article on official postgres documentation.

A window function performs a calculation across a set of table rows that are somehow related to the current row. This is comparable to the type of calculation that can be done with an aggregate function. But unlike regular aggregate functions, use of a window function does not cause rows to become grouped into a single output row — the rows retain their separate identities. Behind the scenes, the window function is able to access more than just the current row of the query result.

In fact, you are doing something like aggregation, but the results are not collapsed by variable aggregation.

Let's consider an example for this page. We have a numeric variable called `some_variable`, on which we will apply various functions. Also, we have a `group` variable that represents the relationship between rows.

In [1]:
docker run -d --rm\
    --name window_functions_example\
    -e POSTGRES_PASSWORD=postgres\
    postgres:15.4 &> /dev/null
sleep 5

docker exec -i window_functions_example psql -U postgres -d postgres << EOF
CREATE TABLE example_table (
    some_value INT NOT NULL,
    "group" TEXT NOT NULL,
    group2 TEXT NOT NULL
);
INSERT INTO example_table (some_value, "group", group2) VALUES
(3, 'C', 'X'),
(3, 'C', 'Y'),
(1, 'C', 'X'),
(4, 'B', 'Y'),
(3, 'A', 'X'),
(1, 'D', 'Y'),
(2, 'A', 'Z'),
(4, 'A', 'X'),
(2, 'A', 'X'),
(0, 'D', 'X'),
(1, 'D', 'Y'),
(4, 'A', 'Y'),
(4, 'B', 'Z'),
(0, 'D', 'X'),
(0, 'D', 'Z'),
(0, 'D', 'X'),
(2, 'D', 'Y'),
(4, 'C', 'Z'),
(4, 'A', 'X'),
(2, 'B', 'X'),
(3, 'A', 'Z'),
(1, 'C', 'Y'),
(3, 'D', 'X'),
(2, 'A', 'X'),
(4, 'B', 'X');

SELECT * FROM example_table LIMIT 5;
EOF

CREATE TABLE
INSERT 0 25
 some_value | group | group2 
------------+-------+--------
          3 | C     | X
          3 | C     | Y
          1 | C     | X
          4 | B     | Y
          3 | A     | X
(5 rows)



**Note** Don't forget to stop the container.

In [20]:
docker stop window_functions_example

window_functions_example


## Syntax

If you are using a window function in the `SELECT` block of your query, you can declare a new output column using the following syntax:

```sql<function> OVER (<relationship>) AS <result name>.```

This allows you to compute the result of the window function and assign it to a new column with the specified `<result name>`.

Functions can be quite different, but actually there are only two components of `<relationship>`: `PARTITION` and `ORDER BY` , so consider them at the beginning.

## Partition

Allows you to define variables that divide rows into groups - rows in each group calculate it function values interpendently.

---

Consider the simplest possible example: for each record, calculate the sum of the `some_value` by the `group` of the row.

In [2]:
docker exec -i window_functions_example psql -U postgres -d postgres << EOF
SELECT 
    some_value, "group",
    SUM(some_value) OVER (PARTITION BY "group") AS result
FROM example_table;
EOF

 some_value | group | result 
------------+-------+--------
          2 | A     |     24
          4 | A     |     24
          3 | A     |     24
          2 | A     |     24
          3 | A     |     24
          4 | A     |     24
          2 | A     |     24
          4 | A     |     24
          4 | B     |     14
          4 | B     |     14
          4 | B     |     14
          2 | B     |     14
          1 | C     |     12
          1 | C     |     12
          4 | C     |     12
          3 | C     |     12
          3 | C     |     12
          1 | D     |      7
          3 | D     |      7
          1 | D     |      7
          0 | D     |      7
          0 | D     |      7
          0 | D     |      7
          0 | D     |      7
          2 | D     |      7
(25 rows)



Now, let's explore a more exotic example to better understand the flexibility of this tool. In this case, for each unique `some_value` we correspond emumeration of the possible values of `group` that ocur with that options of `some_value`.

In [3]:
docker exec -i window_functions_example psql -U postgres -d postgres << EOF
SELECT 
    some_value, "group",
    string_agg("group", ', ') OVER (PARTITION BY some_value) AS result
FROM example_table;
EOF

 some_value | group |       result        
------------+-------+---------------------
          0 | D     | D, D, D, D
          0 | D     | D, D, D, D
          0 | D     | D, D, D, D
          0 | D     | D, D, D, D
          1 | D     | D, C, C, D
          1 | C     | D, C, C, D
          1 | C     | D, C, C, D
          1 | D     | D, C, C, D
          2 | A     | A, A, A, B, D
          2 | A     | A, A, A, B, D
          2 | A     | A, A, A, B, D
          2 | B     | A, A, A, B, D
          2 | D     | A, A, A, B, D
          3 | D     | D, A, C, C, A
          3 | A     | D, A, C, C, A
          3 | C     | D, A, C, C, A
          3 | C     | D, A, C, C, A
          3 | A     | D, A, C, C, A
          4 | B     | B, B, A, A, B, C, A
          4 | B     | B, B, A, A, B, C, A
          4 | A     | B, B, A, A, B, C, A
          4 | A     | B, B, A, A, B, C, A
          4 | B     | B, B, A, A, B, C, A
          4 | C     | B, B, A, A, B, C, A
          4 | A     | B, B, A, A, B, C

### Several variables

When you specify two or more variables within the `PARTITION BY` clause, you create partitions based on a **composite** of those columns.

---

This example demonstrates how to calculate the sum of `some_value` for every unique combination of `group` and `group2`. 

In [8]:
docker exec -i window_functions_example psql -U postgres -d postgres << EOF
SELECT 
    some_value, "group", group2, 
    SUM(some_value) OVER (PARTITION BY "group", group2) AS result
FROM example_table;
EOF

 some_value | group | group2 | result 
------------+-------+--------+--------
          4 | A     | X      |     15
          4 | A     | X      |     15
          2 | A     | X      |     15
          2 | A     | X      |     15
          3 | A     | X      |     15
          4 | A     | Y      |      4
          2 | A     | Z      |      5
          3 | A     | Z      |      5
          2 | B     | X      |      6
          4 | B     | X      |      6
          4 | B     | Y      |      4
          4 | B     | Z      |      4
          1 | C     | X      |      4
          3 | C     | X      |      4
          3 | C     | Y      |      4
          1 | C     | Y      |      4
          4 | C     | Z      |      4
          0 | D     | X      |      3
          3 | D     | X      |      3
          0 | D     | X      |      3
          0 | D     | X      |      3
          1 | D     | Y      |      4
          1 | D     | Y      |      4
          2 | D     | Y      |      4
          

## Order by

The `ORDER BY` clause defines the order of application of the window function. Each unique value of the ordering variable in the aggregation will contain only values corresponding to the previous values according to the order. 

---

The following examples apply to the computation of sums of `some_value` in the order defined by `group`.

In [4]:
docker exec -i window_functions_example psql -U postgres -d postgres << EOF
SELECT 
    some_value, "group",
    SUM(some_value) OVER (ORDER BY "group") AS sv
FROM example_table;
EOF

 some_value | group | sv 
------------+-------+----
          2 | A     | 24
          4 | A     | 24
          3 | A     | 24
          2 | A     | 24
          3 | A     | 24
          4 | A     | 24
          2 | A     | 24
          4 | A     | 24
          4 | B     | 38
          4 | B     | 38
          4 | B     | 38
          2 | B     | 38
          1 | C     | 50
          1 | C     | 50
          4 | C     | 50
          3 | C     | 50
          3 | C     | 50
          1 | D     | 57
          3 | D     | 57
          1 | D     | 57
          0 | D     | 57
          0 | D     | 57
          0 | D     | 57
          0 | D     | 57
          2 | D     | 57
(25 rows)



So for `A` we only have the sum of the `some_values` of the rows corresponding to the `A` group. But for `B` we have the sum on both `A` and `B` groups and so on.

## Partition and order

You can combine `PARTITION BY` and `ORDER BY` within a single `OVER` clause. This allows you to compute aggregations for each distinct combination of variables specified in `PARTITION BY`, while accumulating the aggregation only based on the unique values defined in the `ORDER BY` clause.

---

This example uses `group2` for the `PARTITION BY` clause and `group` for the `ORDER BY` clause within the `OVER` clause. 

In [18]:
docker exec -i window_functions_example psql -U postgres -d postgres << EOF
SELECT 
    some_value, "group", group2,
    SUM(some_value) OVER (PARTITION BY group2 ORDER BY "group") AS sv
FROM example_table;
EOF

 some_value | group | group2 | sv 
------------+-------+--------+----
          2 | A     | X      | 15
          2 | A     | X      | 15
          3 | A     | X      | 15
          4 | A     | X      | 15
          4 | A     | X      | 15
          4 | B     | X      | 21
          2 | B     | X      | 21
          3 | C     | X      | 25
          1 | C     | X      | 25
          0 | D     | X      | 28
          0 | D     | X      | 28
          0 | D     | X      | 28
          3 | D     | X      | 28
          4 | A     | Y      |  4
          4 | B     | Y      |  8
          3 | C     | Y      | 12
          1 | C     | Y      | 12
          2 | D     | Y      | 16
          1 | D     | Y      | 16
          1 | D     | Y      | 16
          3 | A     | Z      |  5
          2 | A     | Z      |  5
          4 | B     | Z      |  9
          4 | C     | Z      | 13
          0 | D     | Z      | 13
(25 rows)



As a result, each unique combination of `group` and `group2` receives its own calculated value.  Within each `group2` value, the `group` values are ordered, and the aggregation accumulates only within that specific `group2` value. This means that different `group2` values have independent sequences for the aggregation. 